In [1]:
using GAMS

In [2]:
using JuMP

In [3]:
using DataFrames

In [4]:
using DataStructures

In [5]:
using Surrogates

In [6]:
using Flux

In [7]:
using Statistics

In [8]:
using GLPK

# Novel mathematical formulation for the short-term scheduling of batch plants: Example 4

This GAMS Model presents an application of the novel mathematical formulation for the short-term scheduling of batch plants. The proposed formulation by Ierapetritou and Floudas  is used on a literature example by Kondili et al that involves 5 tasks and 4 units (heater, 2 reactors and separator). The main process involves a series of 
processing tasks i.e. a heating, 3 reactions and separation.

Formulation Source:
Ierapetritou and Floudas (1998) Effective Continuous-Time Formulation for Short-Term Scheduling. 1. Multipurpose Batch Processes. Independent Engineering Chemical Research 37 (4341-4359)

Example Source:
Kondili E, Pantelides CC, Sargent RWH. (1993) A general algorithm for Short-term scheduling of batch operations. I. MILP formulation. Computer Chemical Engineering. 17 (211–227).


In [9]:
H = 8;
P = 100;

# SETS

Define the sets to be utilised in the MILP
Please task note that if a task can be performed in more than one unit then it is considered a different task for each unit it can be performed in.


In [10]:
I = ["t1","t2","t3","t4","t5","t6","t7","t8"];
J = ["j1","j2","j3","j4"];
N = range(1,P, step = 1);
S = ["s1","s2","s3","s4","s5","s6","s7","s8","s9"];

Jt1 = [J[1]];
Jt2 = [J[2]];
Jt3 = [J[3]];
Jt4 = [J[2]];
Jt5 = [J[3]];
Jt6 = [J[2]];
Jt7 = [J[3]];
Jt8 = [J[4]];

Ij1 = [I[1]];
Ij2 = [I[2],I[4],I[6]];
Ij3 = [I[3],I[5],I[7]];
Ij4 = [I[8]];

Is1 = [I[1]];
Is2 = I[2:3];
Is3 = [I[2],I[3],I[6],I[7]];
Is4 = [I[1],I[4],I[5]];
Is5 = I[4:8];
Is6 = I[2:5];
Is7 = I[6:8];
Is8 = I[4:5];
Is9 = [I[8]];

Ip = I;
Jp = J;
Np = N;



# PARAMETERS
The available information for the process is given below i.e. storage capacity, initial available inventory, batch size and parameters alpha and beta which are the processing time parameters viz. constant and variable terms


In [11]:
vmin = OrderedDict(
   "t1" => 0,
   "t2" => 0,
   "t3" => 0,
   "t4" => 0,
   "t5" => 0,
   "t6" => 0,
   "t7" => 0,
   "t8" => 0
)

vmax = OrderedDict(
   "t1" => 100,
   "t2" => 50,
   "t3" => 80,
   "t4" => 50,
   "t5" => 80,
   "t6" => 50,
   "t7" => 80,
   "t8" => 200,
)

STin = OrderedDict(
     "s1" => 10000,
     "s2" => 10000,
     "s3" => 10000,
     "s4" => 0,
     "s5" => 0,
     "s6" => 0,
     "s7" => 0,
     "s8" => 0,
     "s9" => 0
)

STmax = OrderedDict(
     "s1" => 100000,
     "s2" => 100000,
     "s3" => 100000,
     "s4" => 100,
     "s5" => 200,
     "s6" => 150,
     "s7" => 200,
     "s8" => 100000,
     "s9" => 100000
)

alpha = OrderedDict(
"t1" => 0.667,
"t2" => 1.334,
"t3" => 1.334,
"t4" => 1.334,
"t5" => 1.334,
"t6" => 0.667,
"t7" => 0.667,
"t8" => 1.3342
)

beta = OrderedDict(
"t1" => 0.00667,
"t2" => 0.02664,
"t3" => 0.01665,
"t4" => 0.02664,
"t5" => 0.01665,
"t6" => 0.01332,
"t7" => 0.008325,
"t8" => 0.00666
)

price = OrderedDict(
"s1" => 0,
"s2" => 0,
"s3" => 0,
"s4" => 0,
"s5" => 0,
"s6" => 0,
"s7" => 0,
"s8" => 10,
"s9" => 10
)

demand = OrderedDict(
"s1" => 0,
"s2" => 0,
"s3" => 0,
"s4" => 0,
"s5" => 0,
"s6" => 0,
"s7" => 0,
"s8" => 0,
"s9" => 0
)

#Table p(i,s) 'proportion of state s produced (p(i,s) > 0), consumed (p(i,s) < 0) by task i'
#          s1    s2    s3    s4     s5    s6      s7     s8     s9
#  t1      -1                +1
#  t2           -1   -0.5                 +1
#  t3           -1   -0.5                 +1
#  t4                       -0.4   +0.6   -0.6          +0.4
#  t5                       -0.4   +0.6   -0.6          +0.4
#  t6                -0.2           -1           +1
#  t7                -0.2           -1           +1
#  t8                              +0.1          -1            +0.9


rho_table = DataFrame([(I = "t1", s1 = -1, s2 = 0, s3 = 0, s4 = +1, s5 = 0, s6 = 0, s7 = 0, s8 = 0, s9 = 0 ),
                   (I = "t2", s1 = 0, s2 = -1, s3 = -0.5, s4 = 0, s5 = 0, s6 = 1, s7 = 0, s8 = 0, s9 = 0 ),
                   (I = "t3", s1 = 0, s2 = -1, s3 = -0.5, s4 = 0, s5 = 0, s6 = 1, s7 = 0, s8 = 0, s9 = 0 ),
                   (I = "t4", s1 = 0, s2 = 0, s3 = 0, s4 = -0.4, s5 = +0.6, s6 = -0.6, s7 = 0, s8 = +0.4, s9 = 0 ),
                   (I = "t5", s1 = 0, s2 = 0, s3 = 0, s4 = -0.4, s5 = +0.6, s6 = -0.6, s7 = 0, s8 = +0.4, s9 = 0 ),
                   (I = "t6", s1 = 0, s2 = 0, s3 = -0.2, s4 = 0, s5 = -1, s6 = 0, s7 = +1, s8 = 0, s9 = 0 ),
                   (I = "t7", s1 = 0, s2 = 0, s3 = -0.2, s4 = 0, s5 = -1, s6 = 0, s7 = +1, s8 = 0, s9 = 0 ),
                   (I = "t8", s1 = 0, s2 = 0, s3 = 0, s4 = 0, s5 = +0.1, s6 = 0, s7 = -1, s8 = 0, s9 = +0.9 )]);

rho = OrderedDict( (r[:I],states) => r[Symbol(states)] for r in eachrow(rho_table), states in S);

# MODEL DEFINITION

In [12]:
example4_netprofit = Model(GLPK.Optimzer)

LoadError: [91mUndefVarError: Optimzer not defined[39m

# VARIABLES

In [13]:
@variables example4_netprofit begin
    w[i in I, n in N], Bin
    y[j in J, n in N], Bin
    bm[i in I, j in J, n in N] >= 0
    d[s in S, n in N] >= 0
    ST[s in S, n in N] >= 0
    Ts[i in I, j in J, n in N] >= 0
    Tf[i in I, j in J, n in N] >= 0
end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# ALLOCATION CONSTRAINTS
The allocation constraints express that at each unit j and at an event point n only one of the tasks that can be performed
in this unit (i.e. i ∈ Ij ) should take place

In [14]:
@constraints example4_netprofit begin
    allocation1[j in J, n in N; j == J[1]],
      sum(w[i,n] for i in Ij1) == y[j,n]
    allocation2[j in J, n in N; j == J[2]],
      sum(w[i,n] for i in Ij2) == y[j,n]  
    allocation3[j in J, n in N; j == J[3]],
      sum(w[i,n] for i in Ij3) == y[j,n]
    allocation4[j in J, n in N; j == J[4]],
      sum(w[i,n] for i in Ij4) == y[j,n]  
end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# CAPACITY CONSTRAINTS
The capacity constraints specify the minimum or maximum amount of available material for a task to take place in a unit
Since vmin is zero for all of them, the minimum capacity constraint is not defined


In [15]:
@constraints example4_netprofit begin
    capacity1[i in I, j in Jt1, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity2[i in I, j in Jt2, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity3[i in I, j in Jt3, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity4[i in I, j in Jt4, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity5[i in I, j in Jt5, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity6[i in I, j in Jt6, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity7[i in I, j in Jt7, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity8[i in I, j in Jt8, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]    
end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# STORAGE CONSTRAINTS
This constraint specifies the maximum storage capacity of each material state

In [16]:
@constraints example4_netprofit begin
    storage[s in S, n in N],
      ST[s,n] <= STmax[s]
end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# MATERIAL BALANCE: INITIAL CONDITIONS
This first set constraints is specifying the initial amount of each material state at event point 1


In [17]:
@constraints example4_netprofit begin
    matbal1a[s in S, n in N; s == S[1] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is1 for j in Jt1) - d[s,n]
    matbal1b[s in S, n in N; s == S[2] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is2 for j in Jt2) + sum(rho[i,s]*bm[i,j,n] for i in Is2 for j in Jt3) - d[s,n]
    matbal1c[s in S, n in N; s == S[3] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is3 for j in Jt2) + sum(rho[i,s]*bm[i,j,n] for i in Is3 for j in Jt3) + sum(rho[i,s]*bm[i,j,n] for i in Is3 for j in Jt6) + sum(rho[i,s]*bm[i,j,n] for i in Is2 for j in Jt7) - d[s,n]
    matbal1d[s in S, n in N; s == S[4] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is4 for j in Jt4) + sum(rho[i,s]*bm[i,j,n] for i in Is4 for j in Jt5) - d[s,n]
    matbal1e[s in S, n in N; s == S[5] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is5 for j in Jt6) + sum(rho[i,s]*bm[i,j,n] for i in Is5 for j in Jt7) - d[s,n]
    matbal1f[s in S, n in N; s == S[6] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is6 for j in Jt4) + sum(rho[i,s]*bm[i,j,n] for i in Is6 for j in Jt5) - d[s,n]
    matbal1g[s in S, n in N; s == S[7] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is7 for j in Jt8) - d[s,n]
    matbal1h[s in S, n in N; s == S[8] && n == N[1]],
      ST[s,n] == STin[s] - d[s,n]
    matbal1i[s in S, n in N; s == S[9] && n == N[1]],
      ST[s,n] == STin[s] - d[s,n]
   
end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# MATERIAL BALANCE: PROCESSED
The second set of material balance performs a mass balnce on each state i.e. the amount of material s consumed at event point n by task i in unit j and the amount of material s produced at event point n-1 by task i in unit j


In [18]:
@constraints example4_netprofit begin
    matbal2a[s in S, n in 2:length(N); s == S[1]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt1 for i in Is1) - d[s,n]
    matbal2b[s in S, n in 2:length(N); s == S[2]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt2 for i in Is2) + sum(rho[i,s]*bm[i,j,n] for j in Jt3 for i in Is2) - d[s,n]
    matbal2c[s in S, n in 2:length(N); s == S[3]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt3 for i in Is3) + sum(rho[i,s]*bm[i,j,n] for j in Jt2 for i in Is3) + sum(rho[i,s]*bm[i,j,n] for j in Jt6 for i in Is3) + sum(rho[i,s]*bm[i,j,n] for j in Jt7 for i in Is3 )  - d[s,n]
    matbal2d[s in S, n in 2:length(N); s == S[4]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt4 for i in Is4) + sum(rho[i,s]*bm[i,j,n] for j in Jt5 for i in Is4) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt1 for i in Is4) - d[s,n]
    matbal2e[s in S, n in 2:length(N); s == S[5]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt6 for i in Is5) + sum(rho[i,s]*bm[i,j,n] for j in Jt7 for i in Is5) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt4 for i in Is5) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt5 for i in Is5) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt8 for i in Is5) - d[s,n]
    matbal2f[s in S, n in 2:length(N); s == S[6]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt4 for i in Is6) + sum(rho[i,s]*bm[i,j,n] for j in Jt5 for i in Is6) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt2 for i in Is6) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt3 for i in Is6)  - d[s,n]
    matbal2g[s in S, n in 2:length(N); s == S[7]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt8 for i in Is7) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt6 for i in Is7) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt7 for i in Is7) - d[s,n]
    matbal2h[s in S, n in 2:length(N); s == S[8]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt4 for i in Is8) - d[s,n]
    matbal2i[s in S, n in 2:length(N); s == S[9]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt4 for i in Is9) - d[s,n]
    
end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# DURATION CONSTRAINTS
These constraints don't only specify how long task i will take in unit j but also specifies the dependence of the duration
on the amount of material to be processed by task i in unit j.

In [19]:
@constraints example4_netprofit begin
    duration1[i in I, j in Jt1, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration2[i in I, j in Jt2, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration3[i in I, j in Jt3, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration4[i in I, j in Jt4, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration5[i in I, j in Jt5, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration6[i in I, j in Jt6, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration7[i in I, j in Jt7, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration8[i in I, j in Jt8, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]

end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# SEQUENCE CONSTRAINTS
# i. Same Task in Same Unit
The first set of sequence specify that the start of task i at event point n+1 should start after the end of event point n
for the same task performed in unit j.

In [20]:
@constraints example4_netprofit begin
    sequence1a[i in I, j in Jt1, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] -  H*(2 - w[i,n]- y[j,n])
    sequence1b[i in I, j in Jt2, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1c[i in I, j in Jt3, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1d[i in I, j in Jt4, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1e[i in I, j in Jt5, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] -  H*(2 - w[i,n]- y[j,n])
    sequence1f[i in I, j in Jt6, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1g[i in I, j in Jt7, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1h[i in I, j in Jt8, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])

    sequence2a[i in I, j in Jt1, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2b[i in I, j in Jt2, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2c[i in I, j in Jt3, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2d[i in I, j in Jt4, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2e[i in I, j in Jt5, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2f[i in I, j in Jt6, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2g[i in I, j in Jt7, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2h[i in I, j in Jt8, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]

    sequence3a[i in I, j in Jt1, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3b[i in I, j in Jt2, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3c[i in I, j in Jt3, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3d[i in I, j in Jt4, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3e[i in I, j in Jt5, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3f[i in I, j in Jt6, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3g[i in I, j in Jt7, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3h[i in I, j in Jt8, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]

end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# ii. Different Task in Same Unit
The following constraints establishes the relationship between the starting time of task i at point n+1 and the end time of task i′ (ip) at event point n when different tasks take place in the same unit.

In [21]:
@constraints example4_netprofit begin
    sequence4a[i in Ij1, ip in Ij1, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4b[i in Ij2, ip in Ij2, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4c[i in Ij3, ip in Ij3, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4d[i in Ij4, ip in Ij4, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])

end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# iii. Different Task in Different Unit
When different tasks i and i′ are performed in different units j and j′ but take place one after the other according to the production recipe. These constraints specify the order in which then tasks in each unit should be performed i.e. heating then reaction 1 then reaction 2 (reaction 2 can't be completed until both reaction 1 and heating are completed) then reaction 3 then separation.


In [22]:
@constraints example4_netprofit begin
    sequence5a[i in I, ip in I, j in Jt4, jp in J, n in 1:length(N)-1; i == I[4] && ip == I[1] && jp == J[1]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5b[i in I, ip in I, j in Jt5, jp in J, n in 1:length(N)-1; i == I[5] && ip == I[1] && jp == J[1]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5c[i in I, ip in I, j in Jt4, jp in J, n in 1:length(N)-1; i == I[4] && ip == I[2] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5d[i in I, ip in I, j in Jt5, jp in J, n in 1:length(N)-1; i == I[5] && ip == I[2] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5e[i in I, ip in I, j in Jt4, jp in J, n in 1:length(N)-1; i == I[4] && ip == I[3] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5f[i in I, ip in I, j in Jt5, jp in J, n in 1:length(N)-1; i == I[5] && ip == I[3] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5g[i in I, ip in I, j in Jt6, jp in J, n in 1:length(N)-1; i == I[6] && ip == I[4] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5h[i in I, ip in I, j in Jt7, jp in J, n in 1:length(N)-1; i == I[7] && ip == I[4] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5i[i in I, ip in I, j in Jt6, jp in J, n in 1:length(N)-1; i == I[6] && ip == I[5] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5j[i in I, ip in I, j in Jt7, jp in J, n in 1:length(N)-1; i == I[7] && ip == I[5] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5k[i in I, ip in I, j in Jt8, jp in J, n in 1:length(N)-1; i == I[8] && ip == I[6] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5l[i in I, ip in I, j in Jt8, jp in J, n in 1:length(N)-1; i == I[8] && ip == I[7] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5m[i in I, ip in I, j in Jt6, jp in J, n in 1:length(N)-1; i == I[6] && ip == I[8] && jp == J[4]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5n[i in I, ip in I, j in Jt7, jp in J, n in 1:length(N)-1; i == I[7] && ip == I[8] && jp == J[4]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])

end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# iv. Completion of previous tasks
A task i' (ip) performed in unit j cannot start until task i in unit j is completed


In [23]:
@constraints example4_netprofit begin
   sequence6a[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence6b[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence6c[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence6d[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

   sequence7a[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence7b[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence7c[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence7d[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

   sequence8a[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence8b[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence8c[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence8d[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

   sequence9a[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence9b[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence9c[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence9d[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

   sequence10a[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence10b[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence10c[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence10d[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

   sequence11a[i in I, j in Jt6, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence11b[i in I, j in Jt6, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence11c[i in I, j in Jt6, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence11d[i in I, j in Jt6, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

   sequence12a[i in I, j in Jt7, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence12b[i in I, j in Jt7, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence12c[i in I, j in Jt7, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence12d[i in I, j in Jt7, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

   sequence13a[i in I, j in Jt8, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence13b[i in I, j in Jt8, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence13c[i in I, j in Jt8, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence13d[i in I, j in Jt8, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# TIME HORIZON CONSTRAINTS
Specify that all tasks should task place within the time horizon


In [24]:
@constraints example4_netprofit begin
    timehorizon1[i in I, j in J, n in N],
      Ts[i,j,n] <= H
    timehorizon2[i in I, j in J, n in N],
      Tf[i,j,n] <= H
    
end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# OBJECTIVE FUNCTION

## Profit maximization

In [25]:
@objective example4_netprofit Max begin
    sum(price[s]*d[s,n] for n in N for s in S if (s in [S[8], S[9]]))
end

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

In [26]:
optimize!(example4_netprofit);

LoadError: [91mUndefVarError: example4_netprofit not defined[39m

# Optimal value and schedule

In [27]:
println("Optimal Solution: ", JuMP.objective_value(example4_netprofit))
println("Optimal Schedule: ")
println(JuMP.value.(w))
println(JuMP.value.(bm))
println(JuMP.value.(ts))
println(JuMP.value.(tf))

LoadError: [91mUndefVarError: example4_netprofit not defined[39m